In [1]:
import requests
from bs4 import BeautifulSoup
import os
from tqdm import tqdm
from glob import glob
from datetime import datetime
import dateparser
from unidecode import unidecode
from concurrent.futures import ThreadPoolExecutor, as_completed

In [16]:
cookies = {'cookiesession1': '678B28F695C90B262BFC00B79BBD6CC1',
 'lhc_per': 'vid|a3d85efe963899fee2b5',
 '_gid': 'GA1.3.190366811.1685035962',
 '_ga_3C534XGNC2': 'GS1.1.1685035961.2.1.1685037340.0.0.0',
 '_ga_Q33BVTTDL1': 'GS1.1.1685035961.2.1.1685037340.0.0.0',
 '_gat': '1',
 '_ga_QHEB12PC6D': 'GS1.1.1685035961.2.1.1685037340.0.0.0',
 '_ga': 'GA1.1.2075372866.1684772919'}

headers = {'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
 'Accept-Language': 'en-MY,en;q=0.9,en-US;q=0.8,ms;q=0.7',
 'Cache-Control': 'no-cache',
 'Connection': 'keep-alive',
 'Pragma': 'no-cache',
 'Referer': 'https://ir.uitm.edu.my/view/doclang/2017/',
 'Sec-Fetch-Dest': 'document',
 'Sec-Fetch-Mode': 'navigate',
 'Sec-Fetch-Site': 'same-origin',
 'Sec-Fetch-User': '?1',
 'Upgrade-Insecure-Requests': '1',
 'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36',
 'sec-ch-ua': '"Google Chrome";v="113", "Chromium";v="113", "Not-A.Brand";v="24"',
 'sec-ch-ua-mobile': '?0',
 'sec-ch-ua-platform': '"macOS"'}

In [2]:
import datetime

date1 = datetime.date(2002, 1, 1)
date2 = datetime.date(2023, 1, 1)

def daterange(date1, date2):
    for n in range(int(date2.year) - int(date1.year)+1):
        yield int(date1.year) + n

start_dt = date1
end_dt = date2
dat = []
for dt in daterange(start_dt, end_dt):
    dat.append(dt)
    
dat

[2002,
 2003,
 2004,
 2005,
 2006,
 2007,
 2008,
 2009,
 2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016,
 2017,
 2018,
 2019,
 2020,
 2021,
 2022,
 2023]

In [3]:
urls = [
    'https://eprints.ums.edu.my/view/year/2002.html',
]

In [4]:
all_a = []
for url in urls:
    print(url)
    for y in tqdm(dat):
        splitted = url.split('year/')
        url = splitted[0] + f'year/{y}.html'
        r = requests.get(url)
        soup = BeautifulSoup(r._content, "lxml")
        p = soup.find_all('p')
        a = [p_.find('a') for p_ in p]
        a = [a_.get('href') for a_ in a if a_]
        all_a.extend(a)

https://eprints.ums.edu.my/view/year/2002.html


100%|███████████████████████████████████████████| 22/22 [00:18<00:00,  1.18it/s]


In [6]:
len(all_a)

27168

In [7]:
all_a = sorted(list(set(all_a)))
len(all_a)

27168

In [8]:
import json

with open('all_a-ums.json', 'w') as fopen:
    json.dump(all_a, fopen)

In [9]:
def get_pdf(url):
    try:
        r = requests.get(url, timeout = 10.0, cookies=cookies, headers=headers)
        soup = BeautifulSoup(r._content, "lxml")
        a = soup.find_all('a')
        a = [a_.get('href') for a_ in a]
        a = [a_ for a_ in a if a_ and '.pdf' in a_ and 'haspreview' not in a_]
        a = list(set(a))
        return a
    except:
        return None

get_pdf(all_a[-3])

['https://eprints.ums.edu.my/id/eprint/9997/1/nc0000004634.pdf',
 'https://eprints.ums.edu.my/9997/1.haslightboxThumbnailVersion/nc0000004634.pdf']

In [10]:
!rm -rf pages-ums
!mkdir pages-ums

In [17]:
max_worker = 10

all_a = sorted(all_a)
for i in tqdm(range(0, len(all_a), max_worker)):
    
    filename = os.path.join('pages-ums', f'{i}.json')
    
    if os.path.exists(filename) and os.path.getsize(filename) > 1000:
        continue
    
    pdfs = []
    with ThreadPoolExecutor(max_workers=max_worker) as executor:
        futures = {executor.submit(get_pdf, url): url for url in all_a[i: i + max_worker]}

        for future in as_completed(futures):
            r = future.result()
            if r:
                pdfs.extend(r)
    
    with open(filename, 'w') as fopen:
        json.dump(pdfs, fopen)

100%|███████████████████████████████████████| 2717/2717 [46:20<00:00,  1.02s/it]
